<a href="https://colab.research.google.com/github/sebakc/sebakc/blob/main/gamificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import gradio as gr

# Definición de las preguntas y opciones
questions = [
    {
        "question": "¿Cuáles son tus principales objetivos al usar esta aplicación?",
        "options": ["Mejorar mi productividad", "Alcanzar un estilo de vida más saludable", "Equilibrar el trabajo y el ocio", "Otros (Especificar)"],
        "input_type": "radio"
    },
    {
        "question": "¿Qué actividad te gustaría priorizar para lograr estos objetivos?",
        "options": ["Estudio", "Ejercicio", "Proyectos personales", "Otros (Especificar)"],
        "input_type": "radio"
    },
    {
        "question": "¿Qué te motiva a querer cambiar tus hábitos actuales?",
        "options": ["Lograr más tiempo libre", "Mejorar mi salud", "Incrementar mi productividad", "Reducir el estrés", "Otros (Especificar)"],
        "input_type": "radio"
    },
    {
        "question": "¿Cuántas horas sueles dormir cada día?",
        "input_type": "number"
    },
    {
        "question": "La aplicación ha calculado que te quedan [24 - Horas de sueño] horas disponibles al día. ¿Te gustaría reservar algunas horas específicas para ocio y otras para actividades productivas?",
        "options": ["Sí", "No (la aplicación sugerirá un equilibrio)"],
        "input_type": "radio"
    },
    {
        "question": "Ingresa las tareas que realizas o deseas realizar diariamente y su duración aproximada:",
        "sub_questions": [
            "Trabajo: [Horas/día]",
            "Estudio: [Horas/día]",
            "Hacer ejercicio: [Horas/día]",
            "Limpiar: [Horas/día]",
            "Ver series: [Horas/día]",
            "Ver videos: [Horas/día]",
            "Otras tareas (Especificar): [Horas/día]"
        ],
        "input_type": "text"
    },
    {
        "question": "¿Hay alguna de estas tareas que consideres fundamental realizar todos los días?",
        "options": ["Trabajo", "Estudio", "Hacer ejercicio", "Limpiar", "Otros (Especificar)"],
        "input_type": "radio"
    },
    {
        "question": "¿Te gustaría agregar alguna tarea adicional que creas que pueda ayudarte a alcanzar tus objetivos?",
        "options": ["Meditación", "Lectura", "Cocinar comidas saludables", "Otros (Especificar)"],
        "input_type": "radio"
    },
    {
        "question": "¿Cuántas veces a la semana planeas comer comida chatarra?",
        "input_type": "number"
    },
    {
        "question": "¿Cuántas comidas saludables planeas tener al día?",
        "input_type": "number"
    },
    {
        "question": "¿Te gustaría recibir sugerencias para ajustar tu planificación semanal basadas en tus objetivos?",
        "options": ["Sí", "No"],
        "input_type": "radio"
    },
    {
        "question": "La aplicación generará un resumen con las horas dedicadas a cada actividad, los puntos asignados, y las sugerencias de ajuste según el tiempo disponible. ¿Deseas continuar con esta configuración y comenzar a monitorear tu progreso?",
        "options": ["Sí", "No (Regresar al paso anterior para realizar ajustes)"],
        "input_type": "radio"
    }
]

# Lista para almacenar respuestas del usuario
responses = []

# Índice actual de la pregunta
current_index = 0

def ask_question(radio_response=None, number_response=None, text_response=None):
    global current_index, responses

    # Almacenar la respuesta anterior según el tipo de input
    if current_index > 0:  # Evitar almacenar una respuesta vacía al iniciar
        current_question = questions[current_index - 1]
        if current_question['input_type'] == 'radio' and radio_response is not None:
            if radio_response == "Otros (Especificar)" and text_response:
                responses.append(text_response)  # Guardar el texto personalizado
            else:
                responses.append(radio_response)
        elif current_question['input_type'] == 'number' and number_response is not None:
            responses.append(number_response)
        elif current_question['input_type'] == 'text' and text_response is not None:
            responses.append(text_response)

    # Si hemos alcanzado el final de las preguntas, mostrar las respuestas y habilitar "Reset"
    if current_index >= len(questions):
        summary = "Resumen de tus respuestas:\n"
        for idx, response in enumerate(responses):
            summary += f"Pregunta {idx + 1}: {response}\n"
        return summary, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

    # Obtener la pregunta actual
    current_question = questions[current_index]
    current_index += 1

    # Mostrar la pregunta según su tipo
    if current_question['input_type'] == 'radio':
        return (current_question['question'],
                gr.update(choices=current_question['options'], value=None, visible=True),
                gr.update(visible=False),
                gr.update(value="", visible=False),
                gr.update(visible=False))  # "Reset" oculto
    elif current_question['input_type'] == 'number':
        return (current_question['question'],
                gr.update(visible=False),
                gr.update(value=None, visible=True),
                gr.update(value="", visible=False),
                gr.update(visible=False))  # "Reset" oculto
    elif current_question['input_type'] == 'text':
        return (current_question['question'],
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(value="", visible=True),
                gr.update(visible=False))  # "Reset" oculto

def update_text_input_visibility(radio_value):
    if radio_value == "Otros (Especificar)":
        return gr.update(visible=True)
    else:
        return gr.update(visible=False)

def reset_quiz():
    global current_index, responses
    current_index = 0
    responses = []
    return ask_question()

# Crear la interfaz con Gradio
with gr.Blocks() as demo:
    output_text = gr.Textbox(label="Pregunta", interactive=False)
    radio_choices = gr.Radio(label="Selecciona una opción", interactive=True, visible=False)
    number_input = gr.Number(label="Ingresa un número", visible=False)
    text_input = gr.Textbox(label="Responde aquí", visible=False)

    # Inicializar la primera pregunta
    initial_question, updated_radio, updated_number, updated_text, _ = ask_question()
    output_text.value = initial_question
    radio_choices = gr.Radio(choices=updated_radio['choices'], visible=updated_radio['visible'])
    number_input = gr.Number(visible=updated_number['visible'])
    text_input = gr.Textbox(visible=updated_text['visible'])

    # Botón para enviar la respuesta
    submit_button = gr.Button("Siguiente")
    reset_button = gr.Button("Reset", visible=False)
    radio_choices.change(fn=update_text_input_visibility, inputs=radio_choices, outputs=text_input)

    # Configurar el botón de siguiente y reset
    submit_button.click(ask_question, [radio_choices, number_input, text_input], [output_text, radio_choices, number_input, text_input, reset_button])
    reset_button.click(reset_quiz, [], [output_text, radio_choices, number_input, text_input, reset_button])

# Ejecutar la aplicación
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://47bf3bfe044664448b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
